# Project Data Type

Objectif : trier des données textuelles par type.  
Les types possibles sont :  
2 : code postal  
3 : coordonnées GPS  
4 : adresse  
5 : numéro SIRET  
6 : Année  
7 : Date  
8 : SIREN  
9 : Code NAF/APE  
10 : Autre  
11 : Téléphone  

In [47]:
# Pour manipuler et analyser des données sous formes de tableaux
import pandas as pd
# Pour importer les fichiers CSV, on utilise glob
import glob

In [48]:
# On récupère tous les CSV situés dans le répertoire "./data" du Jupyter notebook
files = glob.glob("./data/*.csv")

# On visualise les colonnes du jeu de données pour voir un peu la tête que ça a
for file in files:
    df = pd.read_csv(file, sep=";")
    df.info()
    # Séparateur pour mieux visualiser les changements de CSV
    print("---------")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13194 entries, 0 to 13193
Data columns (total 29 columns):
Dispositif                                                                   13194 non-null object
Type de structure                                                            13194 non-null object
Année d'agrément                                                             13194 non-null object
Localisation                                                                 10167 non-null object
Début d'agrément                                                             13194 non-null int64
Fin d'agrément                                                               13194 non-null int64
Numéro SIREN                                                                 10107 non-null float64
Nom de l'entreprise                                                          13194 non-null object
Sigle                                                                        4021 non-null object
Ac

In [52]:
# Ici, je vérifie manuellement de quel type semble être la donnée (exemple ici pour le CSV 2)
df = pd.read_csv(files[1], sep=";")
df.sample(5)

,ID,Date,Axe,Départ,Arrivée,Nombre de trains programmés,Nombre de trains ayant circulé,Nombre de trains annulés,Nombre de trains en retard à l'arrivée,Taux de régularité,Nombre de trains à l'heure pour un train en retard
390,INT_3_A,2014-03,Est Centre,BÉZIERS,CLERMONT FERRAND,31.0,31.0,0.0,0.0,100.0,NaN
2032,INT_23_A,2017-01,Est Centre,PARIS EST,TROYES,216.0,214.0,2.0,36.0,83.2,4.9
343,INT_9_A,2014-02,Nord-Normandie,CAEN,PARIS ST LAZARE,176.0,176.0,0.0,15.0,91.5,10.7
1703,INT_23_B,2017-07,Est Centre,TROYES,PARIS EST,238.0,235.0,3.0,29.0,87.7,7.1
387,INT_20_B,2014-02,Est Centre,PARIS BERCY,NEVERS,135.0,134.0,1.0,18.0,86.6,6.4


Dans cet exemple, je ne suis intéressé que par la colone date globalement

In [43]:
# On crée un pseudo-switch qui permet de connaître le type de la donnée en fonction du nom de la colonne du CSV
# Je procède via un grand nombre de "elif" car il n'y a pas de switch/case en Python
# Et que je galère pas mal avec les dictionnaires par manque d'habitude
def returnType(colName):
    # Code postaux
    if colName in ["Code commune (France)"]:
        return 2
    # Coordonnées GPS
    elif colName in ["Géolocalisation", "coord"]:
        return 3
    # Adresses
    elif colName in ["L4_NORMALISEE", "L6_NORMALISEE"]:
        return 4
    # Numéros SIRET
    elif colName in ["SIRETPS"]:
        return 5
    # Années
    elif colName in ["Début d'agrément", "Fin d'agrément"]:
        return 6
    # Dates
    elif colName in ["Date", "JOUR"]:
        return 7
    # Numéros SIREN
    elif colName in ["Numéro SIREN", "SIREN"]:
        return 8
    # Codes NAF/APE
    elif colName in ["Base SIRENE : Code APE de l'établissement siège"]:
        return 9
    # Téléphones
    elif colName in ["TEL", "Téléphone"]:
        return 11
    # Defaut
    else:
        return 0

In [123]:
# On crée une liste temporaire destinée à recevoir les contenus
tempList = []

# Pour chaque CSV, on crée une DataFrame
for file in files:
    print("iterating on file " + file)
    df = pd.read_csv(file, sep=";")
    # Pour chaque colonne du DataFrame actuellement parcouru
    for col in df.columns:
        # On check si la colonne renvoi un type attendu (ou 0 par défaut)
        if returnType(col) != 0:
            print("processing column " + col)
            # Si le type est attendu, on va récupérer tout le contenu de la colonne ainsi que l'indice de type correspondant
            indice = returnType(col)
            valeurs = df[col].tolist()
            # Pour chaque valeur, on va imprimer dans la liste temporaire [indice, valeur]
            for valeur in valeurs:
                tempList.append([indice, str(valeur)])

# Lorsque tous les CSV sont traités, on transforme la liste temporaire en DataFrame
dfsum = pd.DataFrame(tempList, columns=["y", "X"])

iterating on file ./data\fr-esr-cir-et-cii-organismes-et-bureaux-de-style-agrees.csv
processing column Début d'agrément
processing column Fin d'agrément
processing column Numéro SIREN
processing column Téléphone
processing column Géolocalisation
processing column Code commune (France)
processing column Base SIRENE : Code APE de l'établissement siège
iterating on file ./data\regularite-mensuelle-intercites.csv
processing column Date
iterating on file ./data\sirc-17804_9075_61171_2018102_E_Q_20180413_015943916.csv
processing column SIREN
processing column L4_NORMALISEE
processing column SIRETPS
processing column TEL
iterating on file ./data\telecom_qualite_services_mobiles_07-2016.csv
processing column coord
iterating on file ./data\validations-sur-le-reseau-de-surface-nombre-de-validations-par-jour-1er-semestr0.csv
processing column JOUR


In [124]:
# On affiche le contenu d'une 15aine de lignes, on doit voir un peu de tout (dates, coordonnées GPS, adresses, etc)
dfsum.sample(15)

,y,X
199527,3,"44.49918, -0.02585"
228075,3,"46.28793, 4.80895"
72083,2,41159
85150,9,7022Z
45510,11,nan
150604,3,"48.97413, 3.21846"
196281,3,"49.25912, 2.68614"
271148,3,"43.1875, 2.4784"
24462,6,2017
107865,4,261 RUE DE PARIS


A partir de maintenant, je vais utiliser ce dfsum (le DataFrame qui contient la somme traitée de mes CSV) pour entrainer mon machine learning.
Pour des raisons de simplicité je renomme mon dfsum en df.

In [125]:
df = dfsum

## C'est parti pour le machine learning

In [126]:
# On importe une fonction qui convertit des contenus de documents texte en une matrice de comptage de mots/caractères
from sklearn.feature_extraction.text import CountVectorizer

In [127]:
# On détermine qu'un token (fragment de mot) fais la taille d'un mot (à partir de 2 caractères)
vectorizer = CountVectorizer(
    input="content",
    analyzer="word",
    strip_accents=None,
    stop_words=None,
    binary=True,
    ngram_range=(1,2)
)

In [128]:
# On tokenize (on "distingue" les mots/caractères de la donnée)
tokenizer = vectorizer.build_tokenizer()

In [129]:
# On analyse (on distingue des suites de n mots/caractères)
analyzer = vectorizer.build_analyzer()

In [130]:
# On importe une fonction qui segmente les données en jeux d'entrainement et de tests)
from sklearn.model_selection import train_test_split

In [131]:
X = df['X']
y = df['y']

# A l'aide de train_test_split, on va éclater le jeu de données en données d'entrainement (70%) et de test (le reste)
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    y,
    train_size = 0.7
)

In [132]:
# On jette un oeil au format des données d'entrainement de X
df.head()

,y,X
0,6,2012
1,6,2012
2,6,2012
3,6,2012
4,6,2012


In [133]:
# On importe une fonction qui applique des transformations aux données afin d'obtenir une évaluation finale
from sklearn.pipeline import Pipeline
# On importe un classifieur pour les modèles Multinomiaux
from sklearn.naive_bayes import MultinomialNB

In [134]:
# On crée un pipeline de traitement, puis on vectorise les données textuelles et enfin on les classifie
classifier = MultinomialNB()
pipeline = Pipeline(
    [
        ("vectorizer", vectorizer),
        ("classifier", classifier)
    ]
)

In [135]:
# On entraine le classifier sur les données d'entrainement
pipeline.fit(X_train.values, Y_train.values)
# Comme ça peut prendre un petit moment, on imprime un "fit done" à la fin pour avertir de l'arrêt du traitement
print("fit done")

fit done


In [137]:
# On utilise le pipeline entrainé sur les données de test
Y_predicted = pipeline.predict(X_test)

In [138]:
# On regarde les résultats des 10 premières lignes
Y_predicted[1:10]

array([9, 3, 3, 9, 3, 4, 3, 3, 3], dtype=int64)

In [140]:
# On importe une fonction qui calcule le f1_score (le rapport entre la précision et le rappel)
from sklearn.metrics import f1_score
# On calcule le f1_score pour toutes les prédictions effectuées
score = f1_score(
    Y_test,
    Y_predicted,
    average="micro"
)
print("score f1", score)

score f1 0.8625543220085508


Le score f1 du jeu d'entraînement est de 0.8625 soit 86.25% de réussite

In [143]:
# Maintenant on va tester la même chose sur les données qui sont réellement à évaluer (les données de test)
Y_predicted = pipeline.predict(X_train)
# Calcul du score f1
score = f1_score(
    Y_train,
    Y_predicted,
    average="micro"
)
print("score f1", score)

score f1 0.8808202444596928


Le score f1 du jeu de test est de 0.8808 soit 88.08% de réussite

In [ ]:
# Pour s'amuser un peu (et pour voir si les données soumises sont correctement évaluées accessoirement)
# on va permettre de tester directement dans le notebook via un input
dataToTest = input("Insert data to test")
# On "cast" les data à tester en Array via [] car predict attend un iterable
dataType = pipeline.predict([dataToTest])
# On crée un dictionnaire de données pour avoir la correspondance indice/type de donnée
typeDict = {
    2 : "code postal",
    3 : "coordonnées GPS",
    4 : "adresse",
    5 : "numéro SIRET",
    6 : "Année",
    7 : "Date",
    8 : "SIREN",
    9 : "Code NAF/APE",
    10 : "Autre",
    11 : "Téléphone"
}
print("Data type is : " + typeDict.get(int(dataType)))